##classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class','Y_Quality'])
train_y = train_df[['Y_Class','PRODUCT_CODE']]
t_class = train_df[['Y_Class', 'PRODUCT_CODE']]

train_a = train_x[train_x['PRODUCT_CODE'] == 'A_31']
train_t = train_x[train_x['PRODUCT_CODE'] != 'A_31']

t_class_a = t_class[t_class['PRODUCT_CODE'] == 'A_31']
t_class_t = t_class[t_class['PRODUCT_CODE'] != 'A_31']
t_class_a = t_class_a['Y_Class']
t_class_t = t_class_t['Y_Class']

train_a_y = train_y[train_y['PRODUCT_CODE'] == 'A_31']
train_t_y = train_y[train_x['PRODUCT_CODE'] != 'A_31']
train_a_y = train_a_y['Y_Class']
train_t_y = train_t_y['Y_Class']


# train_a.drop(a_na_col, axis = 1, inplace = True)
# train_t.drop(t_na_col, axis = 1, inplace = True)
# train_o.drop(o_na_col, axis = 1, inplace = True)
print(f"Shape of A_31/T_31 : {train_a.shape} {train_t.shape}")

Shape of A_31/T_31 : (249, 2877) (349, 2877)


In [ ]:
col = train_x.columns
na_a_col = []
na_t_col = []
for i in col:
  if train_a[i].count() == 0:
    na_a_col.append(i)
  if train_t[i].count() == 0:
    na_t_col.append(i)

In [ ]:
a_col = []
t_col = []

for i in col:
  if i == 'PRODUCT_CODE' or i == 'LINE':
    continue
  if i not in na_a_col:
    a_col.append(i)
  if i not in na_t_col:
    t_col.append(i)

print(len(a_col), len(t_col))
print(a_col)

2116 677
['X_128', 'X_129', 'X_130', 'X_131', 'X_132', 'X_133', 'X_134', 'X_135', 'X_136', 'X_137', 'X_138', 'X_139', 'X_140', 'X_141', 'X_142', 'X_143', 'X_144', 'X_145', 'X_146', 'X_147', 'X_148', 'X_149', 'X_150', 'X_151', 'X_152', 'X_153', 'X_154', 'X_155', 'X_156', 'X_157', 'X_158', 'X_159', 'X_160', 'X_161', 'X_162', 'X_163', 'X_164', 'X_165', 'X_166', 'X_167', 'X_168', 'X_169', 'X_170', 'X_171', 'X_172', 'X_173', 'X_174', 'X_175', 'X_176', 'X_177', 'X_178', 'X_179', 'X_180', 'X_181', 'X_182', 'X_183', 'X_184', 'X_185', 'X_186', 'X_187', 'X_188', 'X_189', 'X_190', 'X_191', 'X_192', 'X_193', 'X_194', 'X_195', 'X_196', 'X_197', 'X_198', 'X_199', 'X_200', 'X_201', 'X_202', 'X_203', 'X_204', 'X_205', 'X_206', 'X_207', 'X_208', 'X_209', 'X_210', 'X_211', 'X_212', 'X_213', 'X_214', 'X_215', 'X_216', 'X_217', 'X_218', 'X_219', 'X_220', 'X_221', 'X_222', 'X_223', 'X_224', 'X_225', 'X_226', 'X_227', 'X_228', 'X_229', 'X_230', 'X_231', 'X_232', 'X_233', 'X_234', 'X_235', 'X_236', 'X_237', 

In [ ]:
comm_na_col = []
for i in na_a_col:
  if i in na_t_col:
   comm_na_col.append(i)
print(len(comm_na_col))

82


In [ ]:
train_a.drop(comm_na_col, axis = 1, inplace = True)
train_t.drop(comm_na_col, axis = 1, inplace = True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
train_a.drop(t_col, axis = 1, inplace = True)
train_t.drop(a_col,axis = 1, inplace = True)

In [ ]:
print(len(train_a.columns))
print(len(train_t.columns))

2118
679


In [ ]:
train_a = train_a.fillna(0)
train_t = train_t.fillna(0)

In [ ]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']
train_total = [train_a, train_t]
for k in range(2):
  for i in qual_col:
      le = LabelEncoder()
      le = le.fit(train_total[k][i])
      train_total[k][i] = le.transform(train_total[k][i])

      # for label in np.unique(test_x[i]): 
      #     if label not in le.classes_: 
      #         le.classes_ = np.append(le.classes_, label)
      # test_x[i] = le.transform(test_x[i]) 
  print('Done.')

Done.
Done.


In [ ]:
test_a = train_a[200:]
test_a_y = train_a_y[200:]
train_a = train_a[:200]
train_a_y = train_a_y[:200]

test_t = train_t[300:]
test_t_y = train_t_y[300:]
train_t = train_t[:300]
train_t_y = train_t_y[:300]

In [ ]:
class modelA(nn.Module):
  def __init__(self, input_size, output_size, hidden_size):
    super(modelA,self).__init__()

    self.input_size = input_size
    self.output_size = output_size

    self.linear = nn.Linear(input_size,hidden_size)
    self.linear2 = nn.Linear(hidden_size, hidden_size)
    self.fc = nn.Linear(hidden_size, output_size)

    self.softmax = nn.Softmax(dim = 0)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.linear(x)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.relu(out)
    out = self.fc(out)
    return out

In [ ]:

#GPU 체크
is_cuda = torch.cuda.is_available()
if is_cuda:
  device = torch.device("cuda")
  print("GPU is available")
else:
  device = torch.device("cpu")
  print("CPU is availalbe")
     

GPU is available


In [ ]:
# RF_a = RandomForestClassifier(random_state=37).fit(train_a, train_a_y)
# RF_t = RandomForestClassifier(random_state=37).fit(train_t, train_t_y)

# RF_a = RandomForestClassifier(n_estimators = 2000, random_state=37, max_depth = 45).fit(train_a, train_a_y)
# RF_t = RandomForestClassifier(n_estimators = 2000,random_state=37, max_depth = 45).fit(train_t, train_t_y)


# RF_a = DecisionTreeClassifier(random_state=37).fit(train_a, train_a_y)
# RF_t = DecisionTreeClassifier(random_state=37).fit(train_t, train_t_y)

# RF_a = svm.SVC(random_state=37).fit(train_a, train_a_y)
# RF_t = svm.SVC(random_state=37).fit(train_t, train_t_y)

# RF_a = SGDClassifier(random_state=37).fit(train_a, train_a_y)
# RF_t = SGDClassifier(random_state=37).fit(train_t, train_t_y)

# RF_a = LogisticRegression(random_state=37).fit(train_a, train_a_y)
# RF_t = LogisticRegression(random_state=37).fit(train_t, train_t_y)
lr = 0.001
RF_a = modelA(train_a.shape[1],3,(train_a.shape[1]//2)).to(device)
RF_t = modelA(train_t.shape[1],3,(train_t.shape[1]//2)).to(device)
optimizer_a = torch.optim.Adam(RF_a.parameters(), lr = lr)
optimizer_t = torch.optim.Adam(RF_a.parameters(), lr = lr)
cost_a = nn.CrossEntropyLoss().cuda()
cost_t = nn.CrossEntropyLoss().cuda()

print('Done.')

Done.


In [ ]:
# print(list(RF_a.coef_).count(0), len(train_a.columns))
# print(list(RF_t.coef_).count(0), len(train_t.columns))

In [ ]:
train_a_tensorx = torch.FloatTensor(train_a.values).to(device)
train_a_tensory = torch.LongTensor(train_a_y.values).view(-1).to(device)
test_a_tensorx = torch.FloatTensor(test_a.values).to(device)

train_t_tensorx = torch.FloatTensor(train_t.values).to(device)
train_t_tensory = torch.LongTensor(train_t_y.values).view(-1).to(device)
test_t_tensorx = torch.FloatTensor(test_t.values).to(device)

In [ ]:
train_a_tensorx.shape

torch.Size([200, 2118])

In [ ]:
num_epochs = 5000
for epoch in range(num_epochs + 1):
  outputs = RF_a.forward(train_a_tensorx)
  costs = cost_a(outputs, train_a_tensory)

  optimizer_a.zero_grad()
  costs.backward()
  optimizer_a.step()

  if epoch%1000 == 0:
    print(f'Epoch{epoch} : {costs}')



for epoch in range(num_epochs + 1):
  outputs = RF_t.forward(train_t_tensorx)
  costs = cost_t(outputs, train_t_tensory)

  optimizer_t.zero_grad()
  costs.backward()
  optimizer_t.step()
  if epoch%1000 == 0:
    print(f'Epoch{epoch}  : {costs}')

global preds_ac, preds_tc
with torch.no_grad():
  preds_ac = RF_a(test_a_tensorx).detach()
  preds_tc = RF_t(test_t_tensorx).detach()
pred_ac_np = torch.argmax(preds_ac.cpu(), dim = 1).numpy()
pred_tc_np = torch.argmax(preds_tc.cpu(), dim = 1).numpy()
print(accuracy_score(pred_ac_np, test_a_y.values.reshape(-1)))
print(accuracy_score(pred_tc_np, test_t_y.values.reshape(-1)))
print(pred_tc_np)

Epoch0 : 7738105344.0
Epoch1000 : 10.036676406860352
Epoch2000 : 6.0215253829956055
Epoch3000 : 9.518133163452148
Epoch4000 : 2.0313799381256104
Epoch5000 : 28.532896041870117
Epoch0  : 70.06867980957031
Epoch1000  : 70.06867980957031
Epoch2000  : 70.06867980957031
Epoch3000  : 70.06867980957031
Epoch4000  : 70.06867980957031
Epoch5000  : 70.06867980957031
0.3469387755102041
0.16326530612244897
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
# preds_ac = RF_a.predict(test_a)
# preds_tc = RF_t.predict(test_t)
# print('Done.')

In [ ]:
# min_df = 0.525068
# max_df = 0.534843

In [ ]:
# plt.figure(figsize = (10,6))
# plt.scatter(range(len(preds_ac)),preds_ac, label = "pred")
# plt.scatter(range(len(test_a_y)), test_a_y, label = "actual")
# plt.ylim(-0.1,2.1)
# plt.legend()
# plt.show()

In [ ]:
# plt.figure(figsize = (10,6))
# plt.scatter(range(len(preds_tc)),preds_tc, label = "pred")
# plt.scatter(range(len(test_t_y)), test_t_y, label = "actual")
# plt.ylim(-0.1,2.1)
# plt.legend()
# plt.show()

In [ ]:
# print(accuracy_score(preds_ac, test_a_y))
# print(accuracy_score(preds_tc, test_t_y))